<a href="https://colab.research.google.com/github/vixbi/actprobl/blob/main/Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bitsandbytes pymystem3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [13]:
from pymystem3 import Mystem
import re
from collections import Counter
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scipy.spatial.distance import cosine
import torch
import bitsandbytes

# Задание 1.
Проанализируйте текст ниже с помощью майстема. Уберите из получившегося списка элементы, у которых нет грамматического разбора. Далее создайте отдельный список только с частями речи для каждого слова. Превратите этот список отдельных частеречных тэгов в список частеречных триграммов (с пересечением). ['N', "ADJ", "V", "J"] -> [("N", "ADJ", "V"), ("ADJ", "V", ""J")] (тэги не настоящие для примера)

Найдите самый частотный частеречный триграмм. Вставьте триграмм и количество вхождений в форму в таком формате:

N ADJ V;3 (три тэга через пробел и количество вхождение через точку с запятой)

In [46]:
text = """
Античит-движки для игр — это ещё одна крупная область Windows, которая использует драйверы на уровне ядра. Microsoft ведёт переговоры с разработчиками игр о том, как сократить объём использования ядра. Riot Games сообщила Microsoft о готовности следовать потенциальным изменениям безопасности Windows и сократить влияние на ядро ОС.

Летом этого года Microsoft готовится к выпуску обновления Windows, которое будет включать новую функцию Quick Machine Recovery. Функция предназначена для быстрого восстановления системы после сбоя. Ранее Microsoft сообщила о планах сменить синий экран смерти на чёрный.
"""

mystem = Mystem(disambiguation=False)
mystem_raw = mystem.analyze(text)
analysis_list = []
for token in mystem_raw:
    if 'analysis' in token.keys():
        analysis_list.append(token)

analysis_list[0]

{'analysis': [{'lex': 'античит',
   'wt': 0.320835267,
   'qual': 'bastard',
   'gr': 'S,имя,муж,од=им,ед'},
  {'lex': 'античать',
   'wt': 0.3037382897,
   'qual': 'bastard',
   'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'},
  {'lex': 'античить',
   'wt': 0.245045541,
   'qual': 'bastard',
   'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'},
  {'lex': 'античать',
   'wt': 0.04161362302,
   'qual': 'bastard',
   'gr': 'V,сов,нп=непрош,ед,изъяв,3-л'},
  {'lex': 'античать',
   'wt': 0.02966297869,
   'qual': 'bastard',
   'gr': 'V,сов,пе=непрош,ед,изъяв,3-л'},
  {'lex': 'античать',
   'wt': 0.02802527824,
   'qual': 'bastard',
   'gr': 'V,пе=непрош,ед,изъяв,3-л,сов'},
  {'lex': 'античивать',
   'wt': 0.02204770469,
   'qual': 'bastard',
   'gr': 'V,пе=непрош,ед,изъяв,3-л,сов'},
  {'lex': 'античить',
   'wt': 0.00903131762,
   'qual': 'bastard',
   'gr': 'V,сов,нп=непрош,ед,изъяв,3-л'}],
 'text': 'Античит'}

In [57]:
pos_list = []
for analysis in analysis_list:
    for var in analysis['analysis']:
        first_comma = var['gr'].find(',')
        pos = var['gr'][:first_comma]
        if re.fullmatch(r'[A-Za-z]+', pos) == None:
            first_comma = var['gr'].find('=')
            pos = var['gr'][:first_comma]
        pos_list.append(pos)

In [60]:
pos_list

['S',
 'V',
 'V',
 'V',
 'V',
 'V',
 'V',
 'V',
 'S',
 'PR',
 'S',
 'SPRO',
 'PART',
 'APRO',
 'ADV',
 'PART',
 'APRO',
 'ANUM',
 'A',
 'S',
 'APRO',
 'V',
 'S',
 'PR',
 'PART',
 'S',
 'S',
 'V',
 'S',
 'PR',
 'S',
 'S',
 'S',
 'PR',
 'INTJ',
 'S',
 'SPRO',
 'APRO',
 'S',
 'S',
 'S',
 'CONJ',
 'ADVPRO',
 'PART',
 'V',
 'S',
 'V',
 'S',
 'S',
 'V',
 'PR',
 'INTJ',
 'S',
 'S',
 'V',
 'A',
 'S',
 'S',
 'CONJ',
 'INTJ',
 'S',
 'PART',
 'V',
 'S',
 'PR',
 'PART',
 'S',
 'S',
 'S',
 'S',
 'S',
 'ADV',
 'SPRO',
 'APRO',
 'S',
 'V',
 'PR',
 'S',
 'S',
 'S',
 'APRO',
 'ADV',
 'V',
 'V',
 'A',
 'S',
 'S',
 'V',
 'A',
 'PR',
 'A',
 'S',
 'S',
 'PR',
 'ADV',
 'S',
 'S',
 'V',
 'ADV',
 'V',
 'PR',
 'INTJ',
 'S',
 'S',
 'V',
 'A',
 'S',
 'S',
 'PR',
 'PART',
 'A',
 'S',
 'S']

In [66]:
trigrams = list(zip(pos_list, pos_list[1:], pos_list[2:]))
Counter(trigrams).most_common()

[(('S', 'S', 'V'), 6),
 (('S', 'S', 'S'), 6),
 (('V', 'V', 'V'), 5),
 (('A', 'S', 'S'), 5),
 (('V', 'S', 'PR'), 4),
 (('S', 'PR', 'PART'), 3),
 (('S', 'S', 'PR'), 3),
 (('PR', 'INTJ', 'S'), 3),
 (('S', 'V', 'A'), 3),
 (('V', 'A', 'S'), 3),
 (('S', 'PR', 'S'), 2),
 (('PR', 'PART', 'S'), 2),
 (('PART', 'S', 'S'), 2),
 (('S', 'V', 'S'), 2),
 (('PR', 'S', 'S'), 2),
 (('SPRO', 'APRO', 'S'), 2),
 (('S', 'S', 'CONJ'), 2),
 (('PART', 'V', 'S'), 2),
 (('S', 'V', 'PR'), 2),
 (('V', 'PR', 'INTJ'), 2),
 (('INTJ', 'S', 'S'), 2),
 (('S', 'V', 'V'), 1),
 (('V', 'V', 'S'), 1),
 (('PR', 'S', 'SPRO'), 1),
 (('S', 'SPRO', 'PART'), 1),
 (('SPRO', 'PART', 'APRO'), 1),
 (('PART', 'APRO', 'ADV'), 1),
 (('APRO', 'ADV', 'PART'), 1),
 (('ADV', 'PART', 'APRO'), 1),
 (('PART', 'APRO', 'ANUM'), 1),
 (('APRO', 'ANUM', 'A'), 1),
 (('ANUM', 'A', 'S'), 1),
 (('A', 'S', 'APRO'), 1),
 (('S', 'APRO', 'V'), 1),
 (('APRO', 'V', 'S'), 1),
 (('S', 'PR', 'INTJ'), 1),
 (('INTJ', 'S', 'SPRO'), 1),
 (('S', 'SPRO', 'APRO'), 1),
 

# Задание 2.
Вам дана последовательность с правильными ответами для задачи многоклассовой классификации. А также массив с предсказаниями такой же длины, но в виде вероятностей для каждого класса. Для каждой строчки выберете самый вероятный класс.

Рассчитайте TP,FP,TN,FN для каждого класса. Рассчитайте точность для каждого класса по отдельности. Найдите класс с самой высокой точностью и вставьте значение точности для этого класса в форму как ответ. Округлите ответ до 2 знака после запятой (например, 0.12)

*Для этого задания не используйте готовые решения (можно использовать только numpy для работы с массивом вероятностей и встроенные библиотеки вроде collections.Counter для удобства)

In [14]:
multiclass_true = [0,3,0,0,3,0,0,1,2,4,1,2,4,0,2,0,4,1,2,4,1,1,4,1,2,0]

probas = np.array([[0.14283307, 0.69125398, 0.08801677, 0.05890497, 0.01899121],
       [0.05017326, 0.16837714, 0.09787715, 0.37586925, 0.3077032 ],
       [0.07349622, 0.20737874, 0.06666821, 0.0582571 , 0.59419973],
       [0.08704361, 0.18970222, 0.16262277, 0.08630088, 0.47433053],
       [0.16137361, 0.21450292, 0.22870402, 0.24422044, 0.151199  ],
       [0.268094  , 0.43761478, 0.12385365, 0.04917541, 0.12126215],
       [0.27161954, 0.01412896, 0.11056462, 0.09468905, 0.50899783],
       [0.09363269, 0.02693918, 0.33977914, 0.39909705, 0.14055195],
       [0.29520778, 0.05132426, 0.06673668, 0.45254433, 0.13418695],
       [0.15421375, 0.22964685, 0.16662999, 0.21004033, 0.23946907],
       [0.24755032, 0.24628846, 0.12956078, 0.24832473, 0.12827571],
       [0.11797719, 0.03232272, 0.07293219, 0.52455883, 0.25220907],
       [0.75835945, 0.07189134, 0.06032446, 0.04986686, 0.05955789],
       [0.02601848, 0.26673495, 0.03124844, 0.21992134, 0.4560768 ],
       [0.58059452, 0.03148405, 0.11400383, 0.14452069, 0.12939691],
       [0.08215579, 0.32887402, 0.10776154, 0.30880314, 0.17240552],
       [0.11467395, 0.07250172, 0.03868279, 0.04358503, 0.73055651],
       [0.09546111, 0.22318028, 0.04797582, 0.42230537, 0.21107742],
       [0.17088269, 0.31617363, 0.02329777, 0.26480448, 0.22484143],
       [0.1573525 , 0.04416687, 0.59056588, 0.0854817 , 0.12243306],
       [0.16300152, 0.224168  , 0.11143585, 0.09995103, 0.4014436 ],
       [0.17649215, 0.32146966, 0.21575183, 0.0238604 , 0.26242596],
       [0.00860468, 0.01201256, 0.96059244, 0.00327634, 0.01551398],
       [0.06455855, 0.11772163, 0.25971348, 0.32510222, 0.23290411],
       [0.04150647, 0.1329052 , 0.22146507, 0.03930956, 0.5648137 ],
       [0.12671909, 0.01486602, 0.77637796, 0.06239952, 0.01963741]])

preds = list()
for array in probas:
    class_to_proba = dict(enumerate(array))
    proba_to_class = {value: key for key, value in class_to_proba.items()}
    max_proba = max(list(proba_to_class.keys()))
    preds.append(proba_to_class[max_proba])

In [82]:
labels = list(range(5))
dict_tfpn = {key: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0} for key in labels}

for label in labels:
    for i in range(len(preds)):
        if multiclass_true[i] == label and preds[i] == label:
            dict_tfpn[label]['TP'] += 1
        elif multiclass_true[i] != label and preds[i] == label:
            dict_tfpn[label]['FP'] += 1
        elif multiclass_true[i] != label and preds[i] != label:
            dict_tfpn[label]['TN'] += 1
        elif multiclass_true[i] == label and preds[i] != label:
            dict_tfpn[label]['FN'] += 1

In [87]:
def calc_precision(tp, fp):
    return tp/(tp+fp)

for key in dict_tfpn.keys():
    print(f"Точность для класса {key} равна {calc_precision(dict_tfpn[key]['TP'], dict_tfpn[key]['FP'])}")

Точность для класса 0 равна 0.0
Точность для класса 1 равна 0.2
Точность для класса 2 равна 0.0
Точность для класса 3 равна 0.25
Точность для класса 4 равна 0.25


In [81]:
# для проверки
print(classification_report(multiclass_true, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.20      0.17      0.18         6
           2       0.00      0.00      0.00         5
           3       0.25      1.00      0.40         2
           4       0.25      0.40      0.31         5

    accuracy                           0.19        26
   macro avg       0.14      0.31      0.18        26
weighted avg       0.11      0.19      0.13        26



# Задание 3
Загрузите токенизатор и модель sentence-transformers/all-MiniLM-L6-v2. Токенизируйте и рассчитайте векторные представления для двух приведенных ниже текстов. Усредните векторные представления токенов каждого из текстов так, чтобы для каждого из текстов получился вектор размером 384. Рассчитайте косинусное расстояние между двумя этим векторами, округлите его до 2 знака после запятой и вставьте число в форму.

In [15]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

text1 = "Hounds display remarkable fidelity towards their owners, often brightening days through affectionate gestures."
text2 = "Domestic pups exhibit steadfast allegiance to caregivers, frequently lifting spirits via tender behaviors."

# не грузила отдельно токенизатор, потому что encode делает это автоматически,
# но внизу тоже загрузила
embedding1 = model.encode(text1)
embedding2 = model.encode(text2)

cosine_dist = cosine(embedding1, embedding2)

cosine_dist_rounded = round(float(cosine_dist), 2)

cosine_dist_rounded

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.36

In [17]:
# косинусная близость
round(float(cosine_similarity([embedding1], [embedding2])[0][0]), 2)

0.64

In [111]:
# вручную подгруженный токенизатор
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokens1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
tokens2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    embedding1 = model(tokens1)["sentence_embedding"]
    embedding2 = model(tokens2)["sentence_embedding"]

cosine_dist = cosine(embedding1[0].numpy(), embedding2[0].numpy())

cosine_dist_rounded = round(float(cosine_dist), 2)

cosine_dist_rounded

0.36

# Задание 4
Используя модель "HuggingFaceTB/SmolLM2-135M" (huggingface), сгенерируйте продолжение следующего промпта: "I woke up and saw ".

Используйте следующее параметры при генерации: максимальная длина продолжения - 40 токенов, отсутствие повторов длиной 2 токена, отсутствие семплирования (выбирается только самый вероятный токен).

Модель должна быть загружена в 4bit формате.

Вставьте сгенерированный текст в форму целиком (кавычки лучше не включать, но сработает и с ними)


In [1]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceTB/SmolLM2-135M",
    load_in_4bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

prompt = "I woke up and saw "
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    input_ids,
    max_new_tokens=40,
    no_repeat_ngram_size=2,
    do_sample=False,
    num_beams=1,
    early_stopping=True
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [10]:
print(generated_text)

I woke up and saw 12, 23, and 33.  I had to go to the hospital to get the 43 and the other 53 were all in the same hospital.
